In [85]:
class Variable:
    def __init__(self, data):
        self.data=data

In [86]:
import numpy as np

data = np.array(1.0)
x = Variable(data)
x.data

array(1.)

In [87]:
x.data.ndim

0

In [88]:
x.data = np.array([1,2,3])
print(x.data.ndim)
x.data = np.array([[1,2,3],[4,5,6]])
print(x.data.ndim)

1
2


In [89]:
class Function:
    def __call__(self, input):
        x = input.data
        y = x**2
        out = Variable(y)
        return out
    

In [90]:
x = Variable(np.array(10))
f = Function()
y = f(x)
print(type(y))
y.data

<class '__main__.Variable'>


100

In [91]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        out = Variable(y)
        return out
    
    def forward(self, x):
        raise NotImplementedError()

In [92]:
class Square(Function):
    def forward(self, x):
        return x**2

In [93]:
x = Variable(10)
f=Square()
y = f(x)
y.data

100

In [94]:
class Exp(Function):
    def forward(self, x):
        return np.exp(x)

In [95]:
#exp(x^2)^2の計算

s = Square()
e= Exp()
s2 = Square()

x = Variable(0.5)
y = s2(e(s(x)))
print(y.data)

1.648721270700128


In [96]:
#数値微分

def numerical_diff(f,x,eps=1e-4):
    x0 = Variable(x.data-eps)
    x1 = Variable(x.data+eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data-y0.data)/eps/2

In [97]:
#x=2の時のfx=x^2の微分(4.0になる)

f = Square()
x = Variable(2.0)

numerical_diff(f,x)

4.000000000004

In [98]:
#Variableの拡張
class Variable:
    def __init__(self, data):
        self.data=data
        self.grad=None
        
        
#Functionの拡張
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        out = Variable(y)
        self.input = input
        return out
    
#なくても動く
#     def forward(self,x):
#         raise NotImplementedError()
        
#     def backward(self, gy):
#         raise NotImplementedError()

In [99]:
class Square(Function):
    def forward(self, x):
        y = x**2
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = 2*x*gy
        return gx

In [100]:

class Exp(Function):
    def forward(self, x):
        return np.exp(x)
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x)*gy
        return gx

In [101]:
A = Square()
B = Exp()
C = Square()

x = Variable(0.5)
a = A(x)
b = B(a)
y = C(b)
y.data

1.648721270700128

In [102]:
y.grad = np.array(1.0)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)
x.grad

3.297442541400256

In [103]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        f = self.creator  # 1. Get a function
        if f is not None:
            x = f.input  # 2. Get the function's input
            x.grad = f.backward(self.grad)  # 3. Call the function's backward
            x.backward()

class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        out = Variable(y)
        out.set_creator(self)
        self.input = input
        self.out = out
        return out

In [104]:
class Square(Function):
    def forward(self, x):
        y = x**2
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = 2*x*gy
        return gx
class Exp(Function):
    def forward(self, x):
        return np.exp(x)
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x)*gy
        return gx

In [105]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

In [106]:
assert y.creator == C

In [107]:
y.creator.input.creator.input.creator.input == x

True

In [108]:
y.grad = np.array(1.0)
C = y.creator
b = C.input
b.grad = C.backward(y.grad)

In [109]:
B = b.creator
a = B.input
a.grad = B.backward(b.grad)
a.grad

3.297442541400256

In [110]:
A = a.creator
x = A.input
x.grad = A.backward(a.grad)
x.grad

3.297442541400256

In [111]:
#backwardメソッドの追加

class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        f = self.creator
        if f is not None:
            x = f.input
            x.grad = f.backward(self.grad)
            x.backward()
            


In [112]:
class Square(Function):
    def forward(self, x):
        y = x**2
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = 2*x*gy
        return gx

class Exp(Function):
    def forward(self, x):
        return np.exp(x)
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x)*gy
        return gx

In [113]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

In [114]:
y.grad = np.array(1.0)
y.backward()
x.grad

3.297442541400256

In [115]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        funcs = [self.creator]
        while funcs:
#             print(len(funcs))
            f = funcs.pop()
            x,y = f.input, f.out
            x.grad = f.backward(y.grad)
            
            if x.creator is not None:
                funcs.append(x.creator)

In [116]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

In [117]:
y.grad = np.array(1.0)
y.backward()
x.grad

3.297442541400256

In [118]:
def square(x):
    return Square()(x)

def exp(x):
    return Exp()(x)

In [119]:
x = Variable(np.array(0.5))
a = square(x)
b = exp(a)
y = square(b)

y.grad = np.array(0.1)
y.backward()
x.grad

0.3297442541400256

In [120]:
x = Variable(np.array(0.5))
y = square(exp(square(x)))
y.grad = np.array(1)

y.backward()

x.grad

3.297442541400256

In [121]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x,y = f.input, f.out
            x.grad = f.backward(y.grad)
            
            if x.creator is not None:
                funcs.append(x.creator)

In [122]:
x = Variable(np.array(0.5))
y = square(exp(square(x)))
y.backward()
x.grad

3.297442541400256

In [123]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))
        
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x,y = f.input, f.out
            x.grad = f.backward(y.grad)
            
            if x.creator is not None:
                funcs.append(x.creator)

In [124]:
x = Variable(np.array(1.0))
Variable(1.0)#エラーが出る

TypeError: <class 'float'> is not supported

In [125]:
x = np.array([1.0])
x

array([1.])

In [126]:
np.array(1).ndim

0

In [127]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [128]:
np.isscalar(np.float64(1.0))

True

In [129]:
np.isscalar(np.array([1,2,3]))

False

In [130]:
np.isscalar(as_array(np.array([1,2,3])))

False

In [131]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        out = Variable(as_array(y))
        out.set_creator(self)
        self.input = input
        self.out = out
        return out

In [132]:
import unittest

In [138]:
class Square(Function):
    def forward(self, x):
        y = x**2
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = 2*x*gy
        return gx
class Exp(Function):
    def forward(self, x):
        return np.exp(x)
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x)*gy
        return gx

In [149]:
def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(x.data-eps)
    x1 = Variable(x.data+eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) /(eps*2)


class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)
    
    def test_backward(self):
        x = Variable(np.array(3.0))
        y = square(x)
        y.backward()
        expected = np.array(6.0)
        self.assertEqual(x.grad, expected)
        
    def test_gradient_check(self):
        x = Variable(np.random.rand(1))
        y = square(x)
        y.backward()
        num_grad = numerical_diff(square, x)
        flg = np.allclose(x.grad, num_grad)
        self.assertTrue(flg)
        
        
if __name__ == '__main__':
    unittest.main(argv=['ignored', '-v'], exit=False)


test_backward (__main__.SquareTest) ... ok
test_forward (__main__.SquareTest) ... ok
test_gradient_check (__main__.SquareTest) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.009s

OK


TypeError: assertTrue() missing 1 required positional argument: 'expr'